In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, accuracy_score

# Hyperparameters tuning

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
X_train_tumor = pd.read_csv(r"C:\Users\logiusti\Lorenzo\PyWorkspace\Camel-Blue\radiomics_train_tumor_feature.csv")
X_train_no_tumor = pd.read_csv(r"C:\Users\logiusti\Lorenzo\PyWorkspace\Camel-Blue\radiomics_train_no_tumor_feature.csv")

X_valid_tumor = pd.read_csv(r"C:\Users\logiusti\Lorenzo\PyWorkspace\Camel-Blue\radiomics_valid_tumor_feature.csv")
X_valid_no_tumor = pd.read_csv(r"C:\Users\logiusti\Lorenzo\PyWorkspace\Camel-Blue\radiomics_valid_no_tumor_feature.csv")

In [3]:
del X_train_tumor['Image_Name']
del X_train_no_tumor['Image_Name']
del X_valid_tumor['Image_Name']
del X_valid_no_tumor['Image_Name']

In [4]:
X_train = X_train_tumor.append(X_train_no_tumor, ignore_index=True).sample(frac=1)
X_valid = X_valid_tumor.append(X_valid_no_tumor, ignore_index=True)#.sample(frac=1)

In [5]:
y_train = X_train['Label'].to_numpy()
y_valid = X_valid['Label'].to_numpy()

del X_train['Label']
del X_valid['Label']

X_train = X_train.to_numpy()
X_valid = X_valid.to_numpy()

y_train = np.vectorize({'Tumor':1, 'No_Tumor':0}.get)(y_train)
y_valid = np.vectorize({'Tumor':1, 'No_Tumor':0}.get)(y_valid)

# Pick only the most relevant features N:20

In [6]:
X_train_backup = X_train
X_valid_backup = X_valid

In [7]:
X_train = X_train_backup
X_valid = X_valid_backup

In [ ]:

X_train = X_train.transpose()[relev_features].transpose()
X_valid = X_valid.transpose()[relev_features].transpose()


In [ ]:
X_train.shape

In [ ]:


def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = roc_auc_score(y_valid, predictions.round())
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}


def optimize(
             #trials, 
             random_state=1234):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.choice('n_estimators', np.arange(550, 5000, dtype=int)),
        'eta': hp.quniform('eta', 0.15, 1, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 700, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0, 1, 0.005),
        'gamma': hp.quniform('gamma', 0, 1, 0.005),
        'colsample_bytree': hp.quniform('colsample_bytree', 0, 1, 0.005),
        'colsample_bylevel ': hp.quniform('colsample_bylevel ', 0, 1, 0.005),
        'colsample_bynode': hp.quniform('colsample_bynode', 0, 1, 0.005),
        'reg_alpha' : hp.quniform('reg_alpha', 0.003, 1, 0.003),
        'reg_lambda' : hp.quniform('reg_lambda', 0.003, 1, 0.003),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 7,
        'booster': 'gbtree',
        'tree_method': 'gpu_hist',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    trials = Trials()
    best = fmin(score, space, algo=tpe.suggest, 
                trials=trials,
               max_evals=3)
    return best,trials



best_hyperparams,trials = optimize(
                            #trials
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)


Training with params:                                                                                                  
{'booster': 'gbtree', 'colsample_bylevel ': 0.54, 'colsample_bynode': 0.145, 'colsample_bytree': 0.755, 'eta': 0.75, 'eval_metric': 'auc', 'gamma': 0.97, 'max_depth': 567, 'min_child_weight': 1.0, 'n_estimators': 2563, 'nthread': 7, 'objective': 'binary:logistic', 'reg_alpha': 0.21, 'reg_lambda': 0.372, 'seed': 1234, 'silent': 1, 'subsample': 0.16, 'tree_method': 'gpu_hist'}
[0]	eval-auc:0.58021	train-auc:0.68874                                                                                 

[1]	eval-auc:0.62500	train-auc:0.72308                                                                                 

[2]	eval-auc:0.66326	train-auc:0.76281                                                                                 

[3]	eval-auc:0.59526	train-auc:0.78233                                                                                 

[4]	eval-auc:0.585

[63]	eval-auc:0.63377	train-auc:0.82059                                                                                

[64]	eval-auc:0.64664	train-auc:0.82502                                                                                

[65]	eval-auc:0.63908	train-auc:0.82703                                                                                

[66]	eval-auc:0.64785	train-auc:0.82688                                                                                

[67]	eval-auc:0.67904	train-auc:0.83478                                                                                

[68]	eval-auc:0.64767	train-auc:0.82127                                                                                

[69]	eval-auc:0.66381	train-auc:0.82761                                                                                

[70]	eval-auc:0.66912	train-auc:0.83125                                                                                

[71]	eval-auc:0.70068	train-auc:

[130]	eval-auc:0.68551	train-auc:0.84600                                                                               

[131]	eval-auc:0.67475	train-auc:0.84282                                                                               

[132]	eval-auc:0.67475	train-auc:0.84297                                                                               

[133]	eval-auc:0.67868	train-auc:0.84403                                                                               

[134]	eval-auc:0.67729	train-auc:0.84688                                                                               

[135]	eval-auc:0.67837	train-auc:0.84662                                                                               

[136]	eval-auc:0.67964	train-auc:0.84379                                                                               

[137]	eval-auc:0.64857	train-auc:0.83056                                                                               

[138]	eval-auc:0.64144	train-auc

[197]	eval-auc:0.65123	train-auc:0.84576                                                                               

[198]	eval-auc:0.65002	train-auc:0.84535                                                                               

[199]	eval-auc:0.66737	train-auc:0.84783                                                                               

[200]	eval-auc:0.64386	train-auc:0.84409                                                                               

[201]	eval-auc:0.64652	train-auc:0.84737                                                                               

[202]	eval-auc:0.65957	train-auc:0.84838                                                                               

[203]	eval-auc:0.67632	train-auc:0.85285                                                                               

[204]	eval-auc:0.69209	train-auc:0.85301                                                                               

[205]	eval-auc:0.69131	train-auc

[264]	eval-auc:0.71138	train-auc:0.86746                                                                               

[265]	eval-auc:0.71041	train-auc:0.86828                                                                               

[266]	eval-auc:0.70920	train-auc:0.86971                                                                               

[267]	eval-auc:0.71011	train-auc:0.86961                                                                               

[268]	eval-auc:0.71119	train-auc:0.86989                                                                               

[269]	eval-auc:0.69675	train-auc:0.86829                                                                               

[270]	eval-auc:0.68768	train-auc:0.86753                                                                               

[271]	eval-auc:0.69300	train-auc:0.86671                                                                               

[272]	eval-auc:0.69203	train-auc

...

[2274]	eval-auc:0.73719	train-auc:0.92039                                                                              

[2275]	eval-auc:0.73719	train-auc:0.92039                                                                              

[2276]	eval-auc:0.73719	train-auc:0.92041                                                                              

[2277]	eval-auc:0.73549	train-auc:0.92043                                                                              

[2278]	eval-auc:0.73549	train-auc:0.92043                                                                              

[2279]	eval-auc:0.73549	train-auc:0.91792                                                                              

[2280]	eval-auc:0.73549	train-auc:0.92019                                                                              

[2281]	eval-auc:0.74214	train-auc:0.91510                                                                              

[2282]	eval-auc:0.73519	train-au

[2341]	eval-auc:0.73966	train-auc:0.92285                                                                              

[2342]	eval-auc:0.73604	train-auc:0.92408                                                                              

[2343]	eval-auc:0.73604	train-auc:0.92408                                                                              

[2344]	eval-auc:0.73845	train-auc:0.92289                                                                              

[2345]	eval-auc:0.73725	train-auc:0.92448                                                                              

[2346]	eval-auc:0.73725	train-auc:0.92448                                                                              

[2347]	eval-auc:0.73501	train-auc:0.92509                                                                              

[2348]	eval-auc:0.73604	train-auc:0.92398                                                                              

[2349]	eval-auc:0.73604	train-au

[2408]	eval-auc:0.74142	train-auc:0.92402                                                                              

[2409]	eval-auc:0.74142	train-auc:0.92379                                                                              

[2410]	eval-auc:0.73815	train-auc:0.92274                                                                              

[2411]	eval-auc:0.74142	train-auc:0.92385                                                                              

[2412]	eval-auc:0.74142	train-auc:0.92385                                                                              

[2413]	eval-auc:0.74039	train-auc:0.92343                                                                              

[2414]	eval-auc:0.74039	train-auc:0.92343                                                                              

[2415]	eval-auc:0.74057	train-auc:0.92479                                                                              

[2416]	eval-auc:0.74057	train-au

[2475]	eval-auc:0.73694	train-auc:0.92705                                                                              

[2476]	eval-auc:0.74142	train-auc:0.92370                                                                              

[2477]	eval-auc:0.74160	train-auc:0.92349                                                                              

[2478]	eval-auc:0.74142	train-auc:0.92375                                                                              

[2479]	eval-auc:0.73815	train-auc:0.92338                                                                              

[2480]	eval-auc:0.74039	train-auc:0.92201                                                                              

[2481]	eval-auc:0.73779	train-auc:0.92245                                                                              

[2482]	eval-auc:0.74039	train-auc:0.92213                                                                              

[2483]	eval-auc:0.74039	train-au

[2542]	eval-auc:0.74202	train-auc:0.92870                                                                              

[2543]	eval-auc:0.73815	train-auc:0.92763                                                                              

[2544]	eval-auc:0.74202	train-auc:0.92939                                                                              

[2545]	eval-auc:0.74202	train-auc:0.92871                                                                              

[2546]	eval-auc:0.74202	train-auc:0.92871                                                                              

[2547]	eval-auc:0.73960	train-auc:0.92906                                                                              

[2548]	eval-auc:0.74081	train-auc:0.92929                                                                              

[2549]	eval-auc:0.73960	train-auc:0.92997                                                                              

[2550]	eval-auc:0.73960	train-au

In [ ]:
trials

In [ ]:
best_hpy = {'colsample_bylevel ': 0.23500000000000001, 'colsample_bynode': 0.075, 'colsample_bytree': 0.585, 'eta': 0.325, 'gamma': 0.13, 'max_depth': 44, 'min_child_weight': 4.0, 'n_estimators': 4500, 'reg_alpha': 0.24, 'reg_lambda': 0.033, 'subsample': 0.885,
            
            
            'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        
        'nthread': 7,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1}

In [ ]:
clf = xgb.XGBClassifier(gpu_id=0, **best_hpy)


In [ ]:
clf.fit(X_train, y_train, verbose=3)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_valid, y_valid)

In [ ]:
sum(clf.feature_importances_[relev_features])/sum(clf.feature_importances_)

In [ ]:
all_relev_features = clf.feature_importances_

In [ ]:
relev_features = all_relev_features.argsort()[-708:][::-1]

In [ ]:
relev_features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_valid, y_valid)

In [ ]:
clf.feature_importances_

In [ ]:
len(np.where(clf.feature_importances_ == 0)[0])